In [56]:
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry
from bs4 import BeautifulSoup
import pandas as pd
import time

**1. Navigate to https://usapl.liftingdatabase.com/competitions. Using BeautifulSoup, extract information from each competition's website.** 

General notes from the competitions' default database:

- "All" for Type and State has the following HTML: https://usapl.liftingdatabase.com/competitions-default?t=&s=. 
- Changing the Type but keeping State as "All" changes the number after 't ='
    - T =  
        - International = 10 
        - IPF = 5
        - Local = 3
        - NAPF = 7
        - National = 2
        - Pro Meet = 9 
        - Pro Series = 8
        - Regional = 4
        - State = 6

**Code to retrieve results from a competition's url:**

In [67]:
def extract_category(td_tag):
    th_tag_before = td_tag.find_previous('th')  # Find the previous th tag
    th_tag_after = td_tag.find_next('th')  # Find the next th tag

    if th_tag_before and th_tag_after:
        # If the td tag is between two th tags, retrieve the previous th element
        return th_tag_before.get_text().strip()
    elif th_tag_before:
        # If the td tag is after a th tag, retrieve the previous th element
        return th_tag_before.get_text().strip()
    elif th_tag_after:
        # If the td tag is before a th tag, retrieve the next th element
        return th_tag_after.get_text().strip()
    else:
        return None

def retrieve_info(url):
    """
    This function creates a dataframe from the USAPL database.
    """
    retry = Retry(total = 5,
                         backoff_factor = 0.5,
                         status_forcelist = [429, 500, 502, 503, 504])
    adapter = HTTPAdapter(max_retries = retry)
    try:
        response = requests.get(url, timeout = 5)
        if response.status_code == 200:
            soup = BeautifulSoup(response.text)    

            results = []

            # Main html element where data is located 
            content = soup.find('div', id="content")

            if content:
                tables = content.findAll('table')

                if tables: 
                    meet_info = tables[0]
                    meet_results = tables[1]

                    # This is the table with competitor results
                    if meet_results:
                        tr_tags = meet_results.findAll('tr')

                        for tr_tag in tr_tags:
                            td_tags = tr_tag.findAll('td')

                            if len(td_tags) >= 2:
                                category = extract_category(td_tags[0])
                                weight_class = td_tags[0].get_text().strip().replace('-', '')
                                placement = td_tags[1].get_text().strip().replace('.', '')
                                name = td_tags[2].get_text().strip()
                                yob = td_tags[3].get_text().strip()
                                team = td_tags[4].get_text().strip()
                                state = td_tags[5].get_text().strip()
                                lot = td_tags[6].get_text().strip()
                                weight = td_tags[7].get_text().strip()
                                squat_1 = td_tags[8].get_text().strip()
                                squat_2 = td_tags[9].get_text().strip()
                                squat_3 = td_tags[10].get_text().strip()
                                bench_1 = td_tags[11].get_text().strip()
                                bench_2 = td_tags[12].get_text().strip()
                                bench_3 = td_tags[13].get_text().strip()
                                deadlift_1 = td_tags[14].get_text().strip()
                                deadlift_2 = td_tags[15].get_text().strip()
                                deadlift_3 = td_tags[16].get_text().strip()
                                total = td_tags[17].get_text().strip()
                                points = td_tags[18].get_text().strip()
                                drug_tested = td_tags[19].get_text().strip()

                                # Find the previous th tag for the event
                                event_tag = tr_tag.find_previous('th', 'competition_view_event')

                                if event_tag: 
                                    event = event_tag.get_text().strip()

                                else: 
                                    event = None

                                # Create a dictionary for each row
                                meet_results_data = {
                                    'Event': event,
                                    'Category': category,
                                    'Weight Class': weight_class,
                                    'Placement': placement,
                                    'Name': name,
                                    'Year of Birth': yob,
                                    'Team': team,
                                    'State': state,
                                    'Lot': lot,
                                    'Weight': weight,
                                    'Squat 1': squat_1,
                                    'Squat 2': squat_2,
                                    'Squat 3': squat_3,
                                    'Bench Press 1': bench_1,
                                    'Bench Press 2': bench_2,
                                    'Bench Press 3': bench_3,
                                    'Deadlift 1': deadlift_1,
                                    'Deadlift 2': deadlift_2,
                                    'Deadlift 3': deadlift_3,
                                    'Total': total,
                                    'Points': points,
                                    'Drug-Tested': drug_tested
                                }

                                # Append the dictionary to the results list
                                results.append(meet_results_data)

                        # Create a DataFrame from the results list
                        meet_results_df = pd.DataFrame(results)

                    # This is table with meet information
                    if meet_info:
                        tr_tags = meet_info.findAll('tr')

                        # Extracting date, sanction number, state, and meet director information from tr_tags list
                        if len(tr_tags) >= 2:
                            date = tr_tags[0].find('td').get_text(strip=True)
                            sanction_num = tr_tags[1].find('td').get_text(strip=True)
                            
                            # If there are at least 4 elements, extract state and meet director variable 
                            state = tr_tags[2].find('td').get_text(strip=True) if len(tr_tags) >= 3 else None
                            meet_director = tr_tags[3].find('td').get_text(strip=True) if len(tr_tags) >= 4 else None

                        else: 
                            print(f'Error retrieving tr tag in meet_info table element from {url}')

                        # Repeat the values for each row in meet_results_df 
                        meet_info_df = pd.DataFrame({
                            'Meet Date': [date] * len(results),
                            'Sanction Number': [sanction_num] * len(results),
                            'Meet Location': [state] * len(results),
                            'Meet Director': [meet_director] * len(results)
                        }, index= meet_results_df.index)

                        # Concatenate the dataframes along the columns 
                        results_df = pd.concat([meet_info_df, meet_results_df], axis=1)

                    # Retrieve meet name from h3 heading
                    # Create a new column to add to results_df
                    meet_name = content.find('h3').get_text(strip=True)
                    meet_name_df = pd.DataFrame({
                        'Meet Name': [meet_name] * len(results)
                    })

                    # Concatenate the dataframes along the columns 
                    results_df = pd.concat([results_df, meet_name_df], axis = 1)                        

                    #Reorder columns in the DataFrame 
                    column_order = [
                        'Meet Date', 'Sanction Number', 'Meet Location', 'Meet Name', 'Meet Director', 
                        'Event', 'Category', 'Weight Class', 'Placement', 'Name', 
                        'Year of Birth', 'Team', 'State', 'Lot', 'Weight', 
                        'Squat 1', 'Squat 2', 'Squat 3', 'Bench Press 1', 
                        'Bench Press 2', 'Bench Press 3', 'Deadlift 1', 
                        'Deadlift 2', 'Deadlift 3', 'Total', 'Points', 'Drug-Tested'
                    ]
                    results_df = results_df[column_order]
                    
                    print('Success!')
                    return results_df

            else: 
                print(f'Error retrieving table element from {url}')
        else: 
            print(f'Error retrieving {url}')

    except Exception as e:
        print(f'Error retrieving data from {url}: {e}')

**Practice run with definition functions**

In [66]:
url = 'https://usapl.liftingdatabase.com/competitions-view?id=120068'
df = retrieve_info(url)
display(df)

Success!


,Meet Date,Sanction Number,Meet Location,Meet Name,Meet Director,Event,Category,Weight Class,Placement,Name,...,Squat 3,Bench Press 1,Bench Press 2,Bench Press 3,Deadlift 1,Deadlift 2,Deadlift 3,Total,Points,Drug-Tested
0,09/23/2021 - 10/03/2021,IPF,IPF,World Men's Classic Championships,http://goodlift.info/onecompetition_dtl.php?li...,Powerlifting,Male - Raw Junior,74,1,Fujii Yuya,...,-265,160,-170,170,280,295,-300,715,105.22,


**2. Create DataFrame of all competitions and their URL's in USAPL Database.** 

This DataFrame contains information of competition type, date the meet occurred, sanction number, and the direction of the meet. 

In [4]:
list_of_urls = [
    'https://usapl.liftingdatabase.com/competitions-default?t=5&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=3&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=7&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=2&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=9&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=8&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=4&s=',
    'https://usapl.liftingdatabase.com/competitions-default?t=6&s='
]

competition_types = ['IPF', 'Local', 'NAPF', 'National', 
                     'Pro Meet', 'Pro Series', 'Regional', 'State']

all_competitions = []

# Go through list of URL and collect data
for url, competition_type in zip(list_of_urls, competition_types):
    response = requests.get(url)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        # List to collect information from each competition 
        competition_info = []
        
        # Retrieve general information from list of competitions
        tabledata = soup.find('table', 'tabledata')
        if tabledata:
            tr_tags = tabledata.findAll('tr')
            
            for tr_tag in tr_tags:
                td_tags = tr_tag.findAll('td')

                if td_tags:
                    date = td_tags[0].get_text().strip()
                    name = td_tags[1].get_text().strip()
                    sanction_num = td_tags[2].get_text().strip()
                    state = td_tags[3].get_text().strip()
                    
                    # Create a dictionary for each row
                    meet_type_data = {
                        'Date': date,
                        'Name': name,
                        'Meet Type': competition_type,
                        'Sanction Number': sanction_num,
                        'State': state
                    }
                    
                    # Append dictionary to results list
                    competition_info.append(meet_type_data)
                    
        else:
            print(f'Error retrieving URL: {url}')
            
        # Retrieve URL information
        anchor_tags = tabledata.findAll('a')
        url_list = ['https://usapl.liftingdatabase.com/' + anchor_tag.get('href') for anchor_tag in anchor_tags]

        # Update competition_info with Website info
        for url, meet_type_data in enumerate(competition_info):
            meet_type_data['Website'] = url_list[url]

        # Append competition_info to overall list 
        all_competitions.extend(competition_info)

# Create a DataFrame from the results list
competition_type_df = pd.DataFrame(all_competitions)
print('DataFrame created from list of USAPL competitions.')

# Display and save DataFrame as csv
competition_type_df

# competition_type_df.to_csv('../data/USAPL_competitions.csv', index = False)
# print('DataFrame saved as csv file in data folder.')


DataFrame created from list of USAPL competitions.


,Date,Name,Meet Type,Sanction Number,State,Website
0,09/23/2021,World Men's Classic Championships,IPF,IPF,,https://usapl.liftingdatabase.com/competitions...
1,03/08/2020,Arnold SBD Pro American,IPF,USAPL-2020-07,Ohio,https://usapl.liftingdatabase.com/http://goodl...
2,03/07/2020,Arnold Con-Cret Pro Bench Bash,IPF,USAPL-2020-02,Ohio,https://usapl.liftingdatabase.com/competitions...
3,11/18/2019,40th World Open Championships Dubai UAE,IPF,IPF,,https://usapl.liftingdatabase.com/competitions...
4,10/07/2019,World Masters Equipped Championships Potchefst...,IPF,IPF,,https://usapl.liftingdatabase.com/competitions...
...,...,...,...,...,...,...
3250,12/12/2015,2015 USA Powerlifting Idaho State Powerlifting...,State,ID-2015-02,Idaho,https://usapl.liftingdatabase.com/competitions...
3251,12/12/2015,USA Powerlifting South Carolina State Champion...,State,SC-2015-02,South Carolina,https://usapl.liftingdatabase.com/competitions...
3252,12/05/2015,USAPL Colorado State Championships,State,CO-2015-05,Colorado,https://usapl.liftingdatabase.com/competitions...
3253,03/14/2015,SE States BP,State,NS-2015-04,Georgia,https://usapl.liftingdatabase.com/competitions...


**3. Using list of url's from the previous code, retrieve results from each competition using retrieve_info function.**

Use URL's in Website column of competition_type_df to retrieve competition results. 

In [6]:
url_list = competition_type_df
dfs = []

# Iterate through each complete URL
for url in url_list['Website']:
    df = retrieve_info(url)
    dfs.append(df)
    time.sleep(0.5)
    
# Concatenate all DataFrames into one
USAPL_powerlifting_df = pd.concat(dfs, ignore_index=True)

USAPL_powerlifting_df.to_csv('../data/usapl.csv', index = False)
print('DataFrame saved as csv file in data folder.')

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
S

Webscrape errors: 
- Cannot open google spreadsheets (2)
    - https://usapl.liftingdatabase.com/https://docs.google.com/spreadsheets/d/1HnWXQGAxXTpjUVOP9FeEx92E6DcvbtIjXq0DwSqnO8I/edit?usp=sharing
    - https://usapl.liftingdatabase.com/https://docs.google.com/spreadsheets/d/1Al4TiaAWfMddLDETdiRyZIw3eElAlYTEvowd4ugzMcY/edit#gid=0

- 'NoneType' object has no attribute 'get_text' (5)
    - https://usapl.liftingdatabase.com/competitions-view?id=1033:
    - https://usapl.liftingdatabase.com/competitions-view?id=1038:
    - https://usapl.liftingdatabase.com/competitions-view?id=1249:
    - https://usapl.liftingdatabase.com/competitions-view?id=1632:
    - https://usapl.liftingdatabase.com/competitions-view?id=1684:
        - Resolved this issue by adding a None value to Event column in original retrieve_info function

- list index out of range (1) 
    - https://usapl.liftingdatabase.com/competitions-view?id=2011:
        - Used table[2] because table[1] is an 'Attachment' table

- Missing information ('None') in meet_type_data table. 
    - Resolved this issue by editing if statement from 4 to 1 (if len(tr_tags) >= 1). 

- Error retrieving data from https://usapl.liftingdatabase.com/competitions-view?id=1823: HTTPSConnectionPool(host='usapl.liftingdatabase.com', port=443): Max retries exceeded with url: /competitions-view?id=1823 (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x0000028D4B82B650>, 'Connection to usapl.liftingdatabase.com timed out. (connect timeout=None)'))
    - Added timeout to requests call 
    
            response = requests.get(url, timeout = 10)
            
    - Added Retry
    
            retry = Retry(total = 5,
                         backoff_factor = 0.5,
                         status_forcelist = [429, 500, 502, 503, 504])
            adapter = HTTPAdapter(max_retries = retry)
    

**4. There's a major difference in number of competitions in each DataFrame (usapl = 390 vs copm_types = 3255). Find out how to retrieve the other missing data.**

Missing_data_urls DataFrame is a list of urls filtered from the comp_types DataFrame that contained competitions that did not show up in initial webscrape DataFrame. 

In [ ]:
url_list = pd.read_csv('../data/missing_data_urls.csv')
dfs = []

# Iterate through each complete URL
for url in url_list['Website']:
    df = retrieve_info(url)
    dfs.append(df)
    time.sleep(0.5)
    
# Concatenate all DataFrames into one
USAPL_powerlifting_df = pd.concat(dfs, ignore_index=True)

USAPL_powerlifting_df.to_csv('../data/missingdata.csv', index = False)
print('DataFrame saved as csv file in data folder.')


**5. There are meets that showed up in usapl DataFrame with missing information. Repeat webscrape process for this list of meets.**

URL's from these meets were manually extracted from the USAPL competitions website. 

 [1] "IPF World Masters Chamionships"    
 [2] "3rd IPF World Classic Bench Press" 
 [3] "IPF Open World Championships"      
 [4] "2017 NAPF Bench Press Championship"
 [5] "Pacific Invitationals"             
 [6] "Alaska High School Championship"   
 [7] "DCF January Jawn"                  
 [8] "2018 Family Fitness Works Classic" 
 [9] "Big Iron Classic"                  
[10] "Baltimore Open"                    
[11] "Wolf Pack Bench Open"              
[12] "Klubbstevne"                       
[13] "Åpent stevne"                      
[14] "Nasjonalt utstyrsfritt styrkeløft" 
[15] "RM Østafjell"                      
[16] "Arnold"                            
[17] "Arnold - GNC Pro Deadlift"         
[18] "Arnold - Ohio Invitational Ironman"
[19] "Southern Regional Championships"   
[20] "South Dakota State Championships"  

**Practice run to ensure webscrape works:**

In [79]:
url = 'https://usapl.liftingdatabase.com/competitions-view?id=2385'
df = retrieve_info(url)
display(df)

Success!


,Meet Date,Sanction Number,Meet Location,Meet Name,Meet Director,Event,Category,Weight Class,Placement,Name,...,Squat 3,Bench Press 1,Bench Press 2,Bench Press 3,Deadlift 1,Deadlift 2,Deadlift 3,Total,Points,Drug-Tested
0,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Female - Open,84,1,Natalie Richardson,...,,132.5,150,157.5,,,,157.5,694.37,
1,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Female - Raw Master 1a,84+,1,BRENDA KUHNS,...,,85,92.5,97.5,,,,97.5,640.85,
2,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Female - Raw Master 1b,72,1,Karen Olofson,...,,80,-82.5,82.5,,,,82.5,637.31,X
3,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Female - Raw Open,72,-,Karen Olofson,...,,80,-82.5,82.5,,,,82.5,637.31,X
4,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Female - Teen 3,84,1,Natalie Richardson,...,,132.5,150,157.5,,,,157.5,694.37,
5,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Male - Master 3a,105,1,Daryl Krejci,...,,120,127.5,-137.5,,,,127.5,406.30,
6,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Male - Raw Master 3a,105,1,Pete Lien,...,,-130,130,137.5,,,,137.5,483.15,
7,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Male - Raw Master 3a,105,2,Daryl Krejci,...,,107.5,112.5,-117.5,,,,112.5,407.04,
8,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Male - Raw Master 3a,105,3,Arlyn Dyce,...,,-95,100,107.5,,,,107.5,380.28,
9,03/16/2019,SD-2019-01,South Dakota,South Dakota State Championships,None,Bench press,Male - Raw Open,74,1,Rickey Nelsen,...,,137.5,147.5,-152.5,,,,147.5,669.94,


**Update retrieve_info function to retrieve data from table[0] with less than four <td> tags.**

In [80]:
url_list = ['https://usapl.liftingdatabase.com/competitions-view?id=2181',
           'https://usapl.liftingdatabase.com/competitions-view?id=2088',
            'https://usapl.liftingdatabase.com/competitions-view?id=1823',
            'https://usapl.liftingdatabase.com/competitions-view?id=1825',
            'https://usapl.liftingdatabase.com/competitions-view?id=1662',
            'https://usapl.liftingdatabase.com/competitions-view?id=120513',
            'https://usapl.liftingdatabase.com/competitions-view?id=119941',
            'https://usapl.liftingdatabase.com/competitions-view?id=1920',
            'https://usapl.liftingdatabase.com/competitions-view?id=1745',
            'https://usapl.liftingdatabase.com/competitions-view?id=1746',
            'https://usapl.liftingdatabase.com/competitions-view?id=1751',
            'https://usapl.liftingdatabase.com/competitions-view?id=1789',
            'https://usapl.liftingdatabase.com/competitions-view?id=1787',
            'https://usapl.liftingdatabase.com/competitions-view?id=1786',
            'https://usapl.liftingdatabase.com/competitions-view?id=1785',
            'https://usapl.liftingdatabase.com/competitions-view?id=352',
            'https://usapl.liftingdatabase.com/competitions-view?id=353',
            'https://usapl.liftingdatabase.com/competitions-view?id=354',
            'https://usapl.liftingdatabase.com/competitions-view?id=1719',
            'https://usapl.liftingdatabase.com/competitions-view?id=2385'
           ]
dfs = []

# Iterate through each complete URL
for url in url_list:
    df = retrieve_info(url)
    dfs.append(df)
    time.sleep(0.5)
    
# Concatenate all DataFrames into one
USAPL_powerlifting_df = pd.concat(dfs, ignore_index=True)

USAPL_powerlifting_df.to_csv('../data/missinginfo.csv', index = False)
print('DataFrame saved as csv file in data folder.')

Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
Success!
DataFrame saved as csv file in data folder.


**Note:** 
- Some meets have a different format for information in table[0]. Index[2] is Homepage info. Correct this later. 
    - https://usapl.liftingdatabase.com/competitions-view?id=120068 
        - Meet Director info is in Meet Location column. There's no Meet Location. Index[2] is Homepage info. 
    - https://usapl.liftingdatabase.com/competitions-view?id=1789
    - https://usapl.liftingdatabase.com/competitions-view?id=1785